In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D
from sklearn.svm import OneClassSVM
import numpy as np

### Load the pre-trained MobileNetV2 model (without classification head)

In [2]:
base_model = MobileNetV2(weights='imagenet', include_top=False)

9406464/9406464 [==============================] - 0s 0us/step


In [3]:
# Add a global average pooling layer to reduce the spatial dimensions
glb_avg_layer = GlobalAveragePooling2D()(base_model.output)

### Create a feature extraction model using the pre-trained MobileNetV2

In [4]:
feature_extraction_model = tf.keras.Model(inputs=base_model.input, outputs=glb_avg_layer)

In [5]:
# Load your dataset
# Here, we'll use random data for demonstration purposes
num_samples = 1000
input_shape = (224, 224, 3)  # Adjust the input shape based on the MobileNetV2 requirements

# Generate random data for the target class
target_class_data = np.random.random((num_samples, *input_shape))

In [6]:
# Generate random data for the outliers
num_outliers = 100
outlier_data = np.random.random((num_outliers, *input_shape))

### Extract deep features for target class and outliers

In [7]:
target_class_features = feature_extraction_model.predict(target_class_data)
outlier_features = feature_extraction_model.predict(outlier_data)

4/4 [==============================] - 1s 179ms/step


### Train One-Class SVM

In [8]:
# Combine the target class features with some proportion of outlier features to create the training set
proportion_outliers = 0.1
train_features = np.vstack((target_class_features, outlier_features[:int(num_outliers * proportion_outliers)]))

=== np.vstack ===

a = np.array([1, 2, 3])

b = np.array([4, 5, 6])

np.vstack((a,b))

Output: array([[1, 2, 3],
       [4, 5, 6]])

=== np.hstack ===

a = np.array((1,2,3))

b = np.array((4,5,6))

np.hstack((a,b))

Output: array([1, 2, 3, 4, 5, 6])

=======================

a = np.array([[1],[2],[3]])

b = np.array([[4],[5],[6]])

np.hstack((a,b))

Output: array([[1, 4],
       [2, 5],
       [3, 6]])

### Fit the One-Class SVM model

In [9]:
ocsvm = OneClassSVM(gamma='auto')  # You can adjust the hyperparameters as needed
ocsvm.fit(train_features)

OneClassSVM(gamma='auto')

In [10]:
# Test on some new data (you can use a separate test set or additional data)
# For example, using only the outlier data for testing
test_data = outlier_data[int(num_outliers * proportion_outliers):]
test_features = feature_extraction_model.predict(test_data)
predictions = ocsvm.predict(test_features)

3/3 [==============================] - 1s 305ms/step


In [11]:
# In the predictions, -1 indicates outliers, and 1 indicates the target class
print("Predictions:", predictions)

Predictions: [ 1  1  1  1 -1  1 -1 -1  1 -1  1 -1  1 -1  1  1  1  1 -1 -1  1  1  1 -1
 -1 -1 -1  1 -1  1  1  1  1 -1 -1  1  1  1  1  1 -1 -1 -1 -1 -1 -1  1  1
  1 -1 -1 -1  1 -1 -1 -1  1 -1 -1  1 -1  1  1  1 -1  1  1  1  1  1 -1 -1
  1 -1 -1  1  1  1  1 -1  1  1  1 -1  1  1 -1 -1  1  1]
